Chapter 3 Brakcketing
========

3.1 Algorithm - Bracketing an interval in which a local minimum must exist
----

In [19]:
def bracket_minimum(f, x=0, s=1e-2, k=2.0):
    """Find an interval that brackets a minimum of function f."""
    points = [(x, f(x))]  # Track points for visualization
    
    a, ya = x, f(x)
    b, yb = a + s, f(a + s)
    points.append((b, yb))
    
    if yb > ya:
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    
    while True:
        c, yc = b + s, f(b + s)
        points.append((c, yc))
        
        if yc > yb:
            return (a, c) if a < c else (c, a), points
        
        a, ya, b, yb = b, yb, c, yc
        s *= k

In [20]:
# Define the function to minimize
def f(x):
    return (x - 3)**2 + 1

In [21]:
# Create four different starting scenarios
scenarios = [
    {'x': 0, 's': 1e-2, 'k': 2.0, 'title': 'Step 1'},
    {'x': 0, 's': 1e-2, 'k': 2.0, 'title': 'Step 2'},
    {'x': 0, 's': 1e-2, 'k': 2.0, 'title': 'Step 3'},
    {'x': 0, 's': 1e-2, 'k': 2.0, 'title': 'Step 4'}
]

In [22]:
import numpy as np   
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [23]:
### Replicate book figure

In [24]:
fig = make_subplots(rows=1, cols=4, subplot_titles=[s['title'] for s in scenarios])

x_curve = np.linspace(-1, 7, 300)
y_curve = f(x_curve)

# For each scenario, show progressive steps
for idx, scenario in enumerate(scenarios):
    col = idx + 1
    
    # Plot the function curve
    fig.add_trace(
        go.Scatter(x=x_curve, y=y_curve, mode='lines', 
                   line=dict(color='black', width=2),
                   showlegend=False, name='f(x)'),
        row=1, col=col
    )
    
    # Run bracket_minimum and get intermediate points
    (a, c), points = bracket_minimum(f, **{k: v for k, v in scenario.items() if k != 'title'})
    
    # Show only first (idx+2) points to create progression effect
    num_points = min(idx + 2, len(points))
    points_to_show = points[:num_points]
    
    # Plot the evaluation points
    x_points = [p[0] for p in points_to_show]
    y_points = [p[1] for p in points_to_show]
    
    fig.add_trace(
        go.Scatter(x=x_points, y=y_points, mode='markers',
                   marker=dict(color='black', size=10),
                   showlegend=False, name='Evaluations'),
        row=1, col=col
    )
    
    # Add labels for final bracket (a and b/c)
    if num_points >= 2:
        # Find the final bracket points
        final_a = points_to_show[0][0] if num_points == 2 else points_to_show[-3][0]
        final_b = points_to_show[-1][0]
        
        # Determine labels based on order
        if final_a < final_b:
            label_left, label_right = 'a', 'b'
        else:
            label_left, label_right = 'b', 'a'
        
        # # Add annotations below x-axis
        # fig.add_annotation(
        #     x=final_a, y=-0.5, text=label_left,
        #     showarrow=False, font=dict(size=14, style='italic'),
        #     xref=f'x{col}', yref=f'y{col}',
        #     row=1, col=col
        # )
        # fig.add_annotation(
        #     x=final_b, y=-0.5, text=label_right,
        #     showarrow=False, font=dict(size=14, style='italic'),
        #     xref=f'x{col}', yref=f'y{col}',
        #     row=1, col=col
        # )
    
    # Update axes
    fig.update_xaxes(title_text='x', row=1, col=col, range=[-1, 7])
    fig.update_yaxes(title_text='y' if col == 1 else '', row=1, col=col, range=[-1, 10])

# Update layout
fig.update_layout(
    height=300,
    showlegend=False,
    title_text="Bracket Minimum Algorithm Progression",
    title_x=0.5
)

fig.show()

### Interactive version

In [25]:
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

In [ ]:
functions = {
    'Parabola': lambda x: (x - 3)**2 + 1,
    'Sine':     lambda x: np.sin(x) + 2,
    'Quartic':  lambda x: 0.1 * (x - 2)**4 - (x - 2)**2 + 3,
    'Abs':      lambda x: np.abs(x - 2) + 1
}

def plot_bracket_minimum(func_name='Parabola', x_start=0.0, step_size=0.5, 
                         expansion_rate=2.0, step_number=1):
    """Interactive plot of bracket minimum algorithm."""
    
    # Get the function
    f = functions[func_name]
    
    # Run the algorithm
    try:
        (a, c), points = bracket_minimum(f, x=x_start, s=step_size, k=expansion_rate)
    except:
        print("Algorithm failed to converge")
        return
    
    # Limit to requested step number
    step_number = min(step_number, len(points))
    points_to_show = points[:step_number]
    
    # Generate x values for the function curve
    all_x = [p[0] for p in points]
    x_min, x_max = min(all_x), max(all_x)
    margin = (x_max - x_min) * 0.3
    x_curve = np.linspace(x_min - margin, x_max + margin, 500)
    y_curve = f(x_curve)
    
    # Create the figure
    fig = go.Figure()
    
    # Plot the function curve
    fig.add_trace(
        go.Scatter(x=x_curve, y=y_curve, mode='lines',
                   line=dict(color='blue', width=2),
                   name='f(x)')
    )
    
    # Plot all evaluation points
    x_points = [p[0] for p in points_to_show]
    y_points = [p[1] for p in points_to_show]
    
    fig.add_trace(
        go.Scatter(x=x_points, y=y_points, mode='markers',
                   marker=dict(color='red', size=12, symbol='circle',
                               line=dict(color='black', width=2)),
                   name='Evaluated points')
    )
    
    # Add lines connecting evaluated points
    fig.add_trace(
        go.Scatter(x=x_points, y=y_points, mode='lines',
                   line=dict(color='red', width=1, dash='dot'),
                   name='Evaluation sequence',
                   showlegend=True)
    )
    
    # Highlight the bracket if we have at least 3 points
    if step_number >= 3:
        bracket_x = [points_to_show[0][0], points_to_show[-1][0]]
        bracket_y = [points_to_show[0][1], points_to_show[-1][1]]
        
        # Determine labels
        if bracket_x[0] < bracket_x[1]:
            labels = ['a', 'b']
        else:
            labels = ['b', 'a']
        
        # Add vertical lines for bracket
        y_min, y_max = min(y_curve), max(y_curve)
        for i, (bx, label) in enumerate(zip(bracket_x, labels)):
            fig.add_trace(
                go.Scatter(x=[bx, bx], y=[y_min, y_max],
                          mode='lines',
                          line=dict(color='green', width=2, dash='dash'),
                          name=f'Bracket {label}',
                          showlegend=(i == 0))
            )
            
            # Add annotation
            fig.add_annotation(
                x=bx, y=y_min - (y_max - y_min) * 0.1,
                text=f"<b>{label}</b>",
                showarrow=False,
                font=dict(size=16, color='green')
            )
    
    # Add annotations for step info
    info_text = f"Step {step_number}/{len(points)}<br>"
    info_text += f"Points evaluated: {step_number}<br>"
    if step_number >= 3:
        info_text += f"Bracket: [{min(bracket_x):.3f}, {max(bracket_x):.3f}]"
    
    fig.add_annotation(
        text=info_text,
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        showarrow=False,
        font=dict(size=12),
        align="left",
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="black",
        borderwidth=1
    )
    
    # Update layout
    fig.update_layout(
        title=f"Bracket Minimum Algorithm: {func_name}",
        xaxis_title="x",
        yaxis_title="y",
        height=500,
        hovermode='closest',
        showlegend=True
    )
    
    fig.show()

# Create interactive widget
interact(
    plot_bracket_minimum,
    func_name=Dropdown(options=list(functions.keys()), value='Parabola', 
                       description='Function:'),
    x_start=FloatSlider(min=-5, max=5, step=0.5, value=0.0, 
                        description='Start x:'),
    step_size=FloatSlider(min=0.01, max=2.0, step=0.01, value=0.5, 
                          description='Step size:'),
    expansion_rate=FloatSlider(min=1.1, max=3.0, step=0.1, value=2.0, 
                               description='Expansion:'),
    step_number=IntSlider(min=1, max=20, step=1, value=1, 
                          description='Step #:')
);

interactive(children=(Dropdown(description='Function:', options=('Parabola', 'Sine', 'Quartic', 'Abs'), value=…